In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import cv2
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras_tuner as kt
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'D:/DATASET/CNN/fibrosis/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    'D:/DATASET/CNN/fibrosis/val',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use this subset for validation
)


# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

print("Class weights:", class_weights_dict)


Found 5349 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Class weights: {0: 0.29237496583766054, 1: 1.8070945945945946, 2: 3.0653295128939826, 3: 2.996638655462185, 4: 2.7290816326530614}


In [3]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load DenseNet121 with ImageNet weights, excluding the top layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    class_weight=class_weights_dict  # Pass class weights here
)

model.save("D:/DATASET/Models/model_updated_fibrosis.h5")

29084464/29084464 [==============================] - 6s 0us/step


Epoch 1/10
168/168 [==============================] - 601s 4s/step - loss: 1.5889 - accuracy: 0.3958
Epoch 2/10
168/168 [==============================] - 510s 3s/step - loss: 1.2882 - accuracy: 0.5036
Epoch 3/10
168/168 [==============================] - 508s 3s/step - loss: 1.2367 - accuracy: 0.5199
Epoch 4/10
168/168 [==============================] - 525s 3s/step - loss: 1.1729 - accuracy: 0.5337
Epoch 5/10
168/168 [==============================] - 507s 3s/step - loss: 1.1571 - accuracy: 0.5622
Epoch 6/10
168/168 [==============================] - 522s 3s/step - loss: 1.1312 - accuracy: 0.5661
Epoch 7/10
168/168 [==============================] - 526s 3s/step - loss: 1.1114 - accuracy: 0.5695
Epoch 8/10
168/168 [==============================] - 510s 3s/step - loss: 1.1229 - accuracy: 0.5494
Epoch 9/10
168/168 [==============================] - 509s 3s/step - loss: 1.1362 - accuracy: 0.5633
Epoch 10/10
168/168 [==============================] - 507s 3s/step - loss: 1.1177 - accura

In [4]:
from tensorflow.keras.models import load_model

model = load_model('D:/DATASET/Models/model_updated_fibrosis.h5')

In [5]:
import kerastuner as kt
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(hp.Int('units', min_value=512, max_value=1024, step=128), activation='relu')(x)
    x = Dropout(hp.Float('dropout', min_value=0.3, max_value=0.7, step=0.1))(x)
    predictions = Dense(5, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), 
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=10, factor=3, directory='my_dir', project_name='hyperparameter_tuning')
tuner.search(train_generator, epochs=10, validation_data=validation_generator)


Trial 2 Complete [00h 17m 23s]

Best val_accuracy So Far: None
Total elapsed time: 00h 34m 44s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
640               |1024              |units
0.6               |0.3               |dropout
0.01              |0.01              |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
168/168 [==============================] - 519s 3s/step - loss: 1.5610 - accuracy: 0.6560
Epoch 2/2
168/168 [==============================] - 505s 3s/step - loss: 0.9617 - accuracy: 0.6900


Traceback (most recent call last):
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 145, in convert_to_metrics_dict
    best_value, _ = _get_best_value_and_best_epoch_from_history(
  File "C:\Users\JUHI\anaconda3\lib\site-p

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "C:\Users\JUHI\anaconda3\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 145, in convert_to_metrics_dict
    best_value, _ = _get_best_value_a